# 🔧 Fase 02: Data Preprocessing & Master Construction
Este notebook implementa el protocolo de **11 pasos** para transformar los datos raw provenientes de Supabase en un dataset maestro listo para EDA y Modelamiento.

### Protocolo de 11 Pasos:
1. **Estandarización de Contratos**: Validación de esquemas.
2. **Limpieza Estructural**: Eliminación de IDs y columnas técnicas.
3. **Temporal Shielding**: Aplicación de Anti-Leakage (vallas temporales).
4. **Gestión de Duplicados**: Estrategia de `keep_last`.
5. **Transformación de Centinelas**: Conversión de códigos de error a Nulos.
6. **Reindexación de Gaps**: Asegurar continuidad temporal.
7. **Protocolos de Imputación**: Ffill, Bfill y constantes pro-negocio.
8. **Recálculo de la Verdad**: Reconstrucción de variables financieras/unidades.
9. **Cleanup de Columnas de Soporte**: Eliminación de auxiliares anti-pereza.
10. **Agregación y Cruce**: Consolidación mensual (Monthly Resampling).
11. **Generación de Artefactos**: Exportación Parquet y Reporte JSON de Trazabilidad.

In [ ]:
# Celda 1: Setup
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Agregar 'src' al path
sys.path.insert(0, os.path.abspath('..'))

from src.utils.config_loader import load_config
from src.loader import DataLoader
from src.preprocessor import Preprocessor

config = load_config("../config.yaml")

# Redirigir reportes a la carpeta de experimentos si estamos en el notebook (Lab Mode)
config['general']['paths']['reports'] = os.path.join("..", config['general']['paths']['experiments']['phase_02'])

# Configuración de visualización
plt.style.use('ggplot')
%matplotlib inline

print(f"✅ Ambiente de Preprocesamiento configurado (Modo Laboratorio).")
print(f"📂 Los reportes se guardarán en: {config['general']['paths']['reports']}")

In [ ]:
# Celda 2: Carga de Datos Raw
loader = DataLoader()
tables = ["ventas_diarias", "macro_economia", "promocion_dia", "redes_sociales"]
raw_data = {}

for table in tables:
    file_path = os.path.join("..", loader.config["general"]["paths"]["raw"], f"{table}.parquet")
    if os.path.exists(file_path):
        raw_data[table] = pd.read_parquet(file_path)
        print(f"✅ Cargada {table}: {raw_data[table].shape}")
    else:
        print(f"⚠️ No se encontró {table}")

In [ ]:
# Celda 3: Ejecución del Core Preprocessor (11 Pasos)
preprocessor = Preprocessor(config)
master_df = preprocessor.process(raw_data)

print(f"\n✅ Preprocesamiento completado. Master dataset: {master_df.shape}")

## 📈 Auditoría de Trazabilidad

In [ ]:
# Celda 4: Trazabilidad del Proceso
print("📊 DETALLE DE PASOS EJECUTADOS:")
for step, detail in preprocessor.report["steps_detail"].items():
    print(f"\n🔹 {step.upper()}:")
    if isinstance(detail, list):
        for item in detail:
            print(f"   - {item}")
    else:
        print(f"   - {detail}")

## 🏁 Dataset Final

In [ ]:
# Celda 5: Perfil del Dataset Maestro
print(f"Dataset Index: {master_df.index.name}")
print(f"Columnas finales: {list(master_df.columns)}")
display(master_df.describe())
display(master_df.head())

In [ ]:
# Celda 6: Visualización del Target
target_col = config['preprocessing']['target_variable']
plt.figure(figsize=(15, 6))
plt.plot(master_df.index, master_df[target_col], marker='o', linestyle='-', color='darkblue')
plt.title(f"Evolución Mensual de {target_col.replace('_', ' ').title()}")
plt.xlabel("Fecha")
plt.ylabel("Unidades")
plt.grid(True)
plt.show()